<a href="https://colab.research.google.com/github/Param-Bhatt/Search-engine/blob/main/Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
#!pip install pdoc3

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
  Created wheel for pdoc3: filename=pdoc3-0.9.1-cp36-none-any.whl size=115994 sha256=7d29dcdbc94f92c6151df866ff088cf5ef5108615233af5b2ab9e978fafbca4a
  Stored in directory: /root/.cache/pip/wheels/84/0d/9e/b10defcfc3a3b76999ea945f0d57047340456e966e2184a451
Successfully built pdoc3


In [ ]:
!pdoc --html glove.py 

/content/glove.py:11: DeprecationWarning: invalid escape sequence \[
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;!@]')
/usr/local/lib/python3.6/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2020-10-22 13:42:07.437488: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<unknown>:11: DeprecationWarning: invalid escape sequence \[
html/glove.html


In [ ]:
#!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2020-10-22 12:26:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1        6%[>                   ]  49.94M  27.6MB/s               ^C


In [ ]:
#!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import re
import nltk
import numpy as np 
from nltk.corpus import stopwords
from datasets import load_dataset
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;!@]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def init_glove_embeddings(dimentions=300):
    """
    Input: Dimentions of the Embeddings to use\n
    Builds the glove embeddings in a dicyionary from the file\n
    Output: Dictionary of the pretrained dictionary\n
    """
    embeddings_dict = {}
    with open("glove.6B."+str(dimentions)+"d.txt", 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

In [ ]:
def init_dataset():
    """
    Input: None\n
    Builds the dataset\n
    Output: dataset\n
    """
    dataset = load_dataset("aeslc")
    s = dataset['train'][:]['email_body']
    #Emails as 
    arr = []
    for i in range(len(s)):
        arr.append([i for i in s[i].split('\n') if len(i)!=0 ])
    assert 14436 == len(arr)
    return s, arr

In [ ]:
def build_corpus(s):
    """
    Input: An array of each email contiguously\n
    Breaks each email by new line and builds corpus of sentences\n
    Output: sentence corpus\n
    """
    corpus = []
    for email in s:
        email = email.split("\n")
        #print(email)
        p=[]
        #print()
        for i in range(len(email)):
            if len(email[i]) > 0:
                #prep_s = preprocess(email[i])
                prep_s = email[i]
                if len(prep_s) > 0:
                    p.append(prep_s)
        corpus.append(p)
    return corpus

In [ ]:
def preprocess(text):
    """
    Input: a string\n
    Takes text as input and\n
        1) Makes everything lower case.\n
        2) Deletes '[/(){}\[\]\|@,;!@]' and then '[^0-9a-z #+_]'.\n
        3) Removes StopWords.\n
        4) Lemmatizes every word.\n
    Output: preprocessed string\n
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(" " , text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub("", text)         # delete symbols which are in BAD_SYMBOLS_RE from text
    textlist = text.split()
    text = [wrd for wrd in textlist if wrd not in STOPWORDS]
    text = " ".join(text)
    
    #Lemmatize
    lem_words = []
    for word in text.split():
        lem_words.append(lemmatizer.lemmatize(word))
    return " ".join(lem_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Function

In [ ]:
def glove_embed(sent):
    """
    Input: Sentence as string\n
    Finds the glove embeddings of each word and adds it for each sentence\n
    Output: Embeddings and the length of words taken to form the embeddings\n
    """
    sent = sent.lower()
    embeddings = [0 for _ in embeddings_dict["king"]]
    length = 0
    for word in sent.split():
        try:
            #print(word)
            embeddings+=embeddings_dict[word]
            length+=1
        except:
            #print("ERROR", word)
            pass
            
    return embeddings, length

In [ ]:
def build_index(corpus):
    """
    Input: Corpus\n
    Builds the index of embeddings of each sentence in the corpus\n
    Output: Embddeings Index of each sentence in the corpus\n
    """
    embeddings = []
    sentence_corpus = []
    for email in range(len(corpus)):
        printc=True
        #print("EMAIL", corpus[email])
        for i in corpus[email]:
            #print(i)
            sentence_corpus.append(i)
            j, length = glove_embed(i)
            #print(length)
            if length<= 0:
                #print("Sent not found",  i)
                embeddings.append(None)
                continue
            if email%1000==0 and printc:
                print("Iteration: ", email)
                printc=False
            try:
                embeddings.append(j/length)
            except:
                print(i, email)
    assert len(embeddings) == len(sentence_corpus)
    return embeddings, sentence_corpus


In [ ]:
def cosine_score(vec1, vec2):
    """
    Input: Two embeddings of same dimentions\n
    Finds the cosine score among them\n
    Output: Cosine Similarity\n
    """
    try:
        if vec1.all() == None or vec2.all() == None:
            return 0
    except: 
        return 0
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def search(query, sentence_corpus, top_n=2):
    """
    Input: query to search, Sentence corpus to search in , top n results\n
    Searches with cosing similarity among the embeddings\n
    Output: most similar items to query\n
    """
    q_emb = []
    j, length = glove_embed(query.lower())
    if length<= 0:
        print("Sent not found",  i)
        q_emb.append(None)
    try:
        q_emb.append(j/length)
    except:
        print(i, email)
    similarity = []
    for emb in range(len(embeddings)):
        score = cosine_score(embeddings[emb], q_emb[0])
        #print(emb, cosine_score(embeddings[emb], q_emb[0]))
        similarity.append(cosine_score(embeddings[emb], q_emb[0]))
    sim = np.argsort(similarity)[::-1][:top_n]
    print("Query:", query)
    print(top_n)
    for index in range(top_n):
        print("Cosine Score: {:.3f}, {}".format(similarity[sim[index]], sentence_corpus[sim[index]]))


In [ ]:
%%time

print("Building Glove Dictionary...")
embeddings_dict = init_glove_embeddings()
print("Built Glove Dictionary")

print("Initializing dataset...")
s, arr = init_dataset()
print("Initialized Dataset")

print("Building corpus...")
corpus = build_corpus(s)
print("Built corpus")

print("Building index...")
embeddings, sentence_corpus = build_index(corpus)
print("Built index")

query = corpus[0][2]
print("Searching Query:", query)
search(query, sentence_corpus)

Building Glove Dictionary...
Built Glove Dictionary
Initializing dataset...


Using custom data configuration default
Reusing dataset aeslc (/root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96)


Initialized Dataset
Building corpus...
Built corpus
Building index...
Iteration:  0
Iteration:  1000
Iteration:  2000
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000
Iteration:  10000
Iteration:  11000
Iteration:  12000
Iteration:  13000
Iteration:  14000
Built index
Searching Query: I will fill in the Legal description of the property one I have received it.
Query: I will fill in the Legal description of the property one I have received it.
2
Cosine Score: 1.000, I will fill in the Legal description of the property one I have received it.
Cosine Score: 0.959, I know that an issue has been raised about payment of incentives to employees on the North American payroll, but we have not received further clarification on the matter, so I would like to send you the "Document" which lists out who is included (by location and payroll) and what the definition of the Scheme is.
CPU times: user 44.5 s, sys: 1.54 s, total: 46 s

In [ ]:
%%time
search(query, sentence_corpus)
print()

Query: I will fill in the Legal description of the property one I have received it.
2
Cosine Score: 1.000, I will fill in the Legal description of the property one I have received it.
Cosine Score: 0.959, I know that an issue has been raised about payment of incentives to employees on the North American payroll, but we have not received further clarification on the matter, so I would like to send you the "Document" which lists out who is included (by location and payroll) and what the definition of the Scheme is.

CPU times: user 3.77 s, sys: 3.78 ms, total: 3.78 s
Wall time: 3.79 s
